![Banner](./img/AI_Special_Program_Banner.jpg)  

# Aggregation of data and pivot tables II - Material
---

In this learning unit, we want to look further into **reshaping** of data - in particular, we want to look at the principle of **tidy data**, which was coined by R developer [Hadley Wickham 2013](https://www.jstatsoft.org/article/view/v059i10). With [tidyverse](https://www.tidyverse.org/), there is an entire ecosystem for R that is geared towards tidy data. However, with the flexibility of Python and Pandas, we can also convert our data into a tidy format without too much effort.

Tidy data is often mentioned in connection with data cleaning. Strictly speaking, however, it would be more appropriate to assign it to *Data Arrangement* or *Data Reshaping*.

## Table of contents
---

- [Tidy Data Principles](#Tidy-Data-Principles)
    - [Terminology](#Terminology)
    - [Principles](#Principles)
- [Tidying of messy data](#Tidying-of-messy-data)
    - [Column names are values and not variables](#Column-names-are-values-and-not-variables)
    - [Several variables are stored in one column](#Several-variables-are-stored-in-one-column)
    - [Variables are saved in rows and columns](#Variables-are-saved-in-rows-and-columns)
    - [Several types of observations are saved in the same table](#Several-types-of-observations-are-saved-in-the-same-table)
    - [A single type of observation is stored in several tables](#A-single-type-of-observation-is-stored-in-several-tables)
- [Extract observations from lists with explode()](#Extract-observations-from-lists-with-explode())
- [Learning outcomes](#Learning-outcomes)

## Tidy Data Principles
---

![Tidy Data](img/tidy_data.png)
Source: [R for Data Science](https://r4ds.had.co.nz/) (Hadley Wickham & Garrett Grolemund)

Tidy data refers to a specific structure in which data should be available so that it can be analyzed and manipulated as easily as possible. Such *standardization* of the data structure also has the advantage that it can be repeated for all types of data. This enables analysis and visualization with the same tools.

### Terminologies

In order to understand the Tidy Data principles, a number of terms (*Data Semantics*) need to be explained. A dataset is a collection of **values**, which can be either numeric or categorical in nature. These values can always be assigned to a **variable** and an **observation**. A variable contains all values that measure one and the same underlying attribute (such as altitude, temperature, duration). An observation contains all values that are measured in the same unit (such as a person or a day) across different attributes.

Let's illustrate this further with an example:

In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.DataFrame.from_dict({'person' : ['John Smith', 'Jane Doe', 'Mary Johnson'],
              'treatmentA' : [np.nan, 16, 3],
              'treatmentB' : [2, 11, 1]})

,person,treatmentA,treatmentB
0,John Smith,NaN,2
1,Jane Doe,16.0,11
2,Mary Johnson,3.0,1


This dataset contains 18 values, which are represented by 3 variables and 6 observations. The variables are:

1. `person`, with the three values John, Mary and Jane.
2. `treatment`, with the two values a and b.
3. `result`, with five or six values, depending on how the missing value is recorded (NaN, 16, 3, 2, 11, 1).

It is not always immediately clear what are observations and what are variables in a dataset. This depends on the structure of the data and also the underlying question of the analysis. For example, two variables could be used for the columns 'phone_private' and 'phone_work'. However, if it is a question of *fraud detection*, the variables would probably be selected as `phone_number` and `phone_type`, as the use of the same telephone number by several people could be an indicator of fraud.

### Principles

Equipped with these concepts, we can now understand the three principles of Tidy Data:

1. *Each variable forms a column*
2. *Each observation forms a row*
3. *Each type of observational unit forms a table*

These principles are closely related to [Boyce-Codd normal form](https://en.wikipedia.org/wiki/Boyce%E2%80%93Codd_normal_form). If data is available in Tidy Data format, certain analyses and visualizations can often be carried out much faster and more intuitively. In the following, we will look at common violations of tidy data principles in data and how these can be remedied with the help of pandas.

## Tidying of messy data
---

Unfortunately, real data violating the principles of tidy data is not the exception, but the rule. The following problems frequently occur in messy data (original wording from [Wickham 2013](https://vita.had.co.nz/papers/tidy-data.pdf)):

* Column headers are values, not variables
* Multiple variables are stored in one column
* Variables are stored in both rows *and* columns
* Multiple types of observational units are stored in the same table
* A single observational unit is stored in multiple tables

In the following, we will look at the different types of this messy data and "clean it up" in relation to Tidy Data. Specifically, an implementation in Pandas is presented - the data was selected analogous to the paper by [Hadley Wickham](https://vita.had.co.nz/papers/tidy-data.pdf), so that it is possible to trace the processes along the paper exactly.

### Column headers are values and not variables

A common type of messy dataset is tabular data designed for presentation, where variables form both the rows and the columns and column headers are values rather than variable names. This is the case in our initial example (patient data). The following data also exhibits such a violation. It is a survey by the [Pew Research Center](https://www.pewresearch.org/download-datasets/) on the relationship between income and religious affiliation in the USA.

In [3]:
df_relinc = pd.read_csv('./data/relinc.csv')

In [4]:
df_relinc

,religion,<10k,10-20k,20-30k,30-40k,40-50k,50-75k,75-100k,100-150k,>150k,refused
0,Agnostic,27,34,60,81,76,137,122,109,84,96
1,Atheist,12,27,37,52,35,70,73,59,74,76
2,Buddhist,27,21,30,34,33,58,62,39,53,54
3,Catholic,418,617,732,670,638,1116,949,792,633,1489
4,refused,15,14,15,11,10,35,21,17,18,116
5,Evangelical Prot,575,869,1064,982,881,1486,949,723,414,1529
6,Hindu,1,9,7,9,11,34,47,48,54,37
7,Historically Black Prot,228,244,236,238,197,223,131,81,78,339
8,Jehovah's Witness,20,27,24,24,21,30,15,11,6,37
9,Jewish,19,19,25,25,30,95,69,87,151,162


This data has 3 variables: `religion`, `income` and `frequency`. It is easy to see that the column names here are values of the variable `income`. This violates the *first principle*, according to which each variable should form a column. To put the data into a tidy format, Pandas offers the function [`pd.melt()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.melt.html):

```python
pd.DataFrame.melt(id_vars=None, value_vars=None, var_name=None, value_name='value', col_level=None, ignore_index=True)
```

* `id_vars`: Identifying variable(s)
* `value_vars`: Columns which are "melted". If nothing is specified, all columns except `id_vars` are used
* `var_name`: Name of the `variable` column
* `value_name`: Name of the `value` column
* `ignore_index`: If `False`, the original index is retained (and probably repeated often)

In [5]:
df_melt_relinc = pd.melt(df_relinc, id_vars=['religion'], var_name='income', value_name='frequency');df_melt_relinc[:20]

,religion,income,frequency
0,Agnostic,<10k,27
1,Atheist,<10k,12
2,Buddhist,<10k,27
3,Catholic,<10k,418
4,refused,<10k,15
5,Evangelical Prot,<10k,575
6,Hindu,<10k,1
7,Historically Black Prot,<10k,228
8,Jehovah's Witness,<10k,20
9,Jewish,<10k,19


This has made our table considerably longer:

In [6]:
df_melt_relinc.shape

(180, 3)

The different formats are therefore often referred to as [wide format and long format](https://en.wikipedia.org/wiki/Wide_and_narrow_data).

#### Connection to the pivot table

Pandas also refers to the process of melting as *unpivoting*. In fact, our data in tidy format can be returned to its original form with `pivot_table()`:

In [7]:
df_melt_relinc.pivot_table(index='religion', columns='income', values='frequency')

income,10-20k,100-150k,20-30k,30-40k,40-50k,50-75k,75-100k,<10k,>150k,refused
religion,,,,,,,,,,
Agnostic,34.0,109.0,60.0,81.0,76.0,137.0,122.0,27.0,84.0,96.0
Atheist,27.0,59.0,37.0,52.0,35.0,70.0,73.0,12.0,74.0,76.0
Buddhist,21.0,39.0,30.0,34.0,33.0,58.0,62.0,27.0,53.0,54.0
Catholic,617.0,792.0,732.0,670.0,638.0,1116.0,949.0,418.0,633.0,1489.0
Evangelical Prot,869.0,723.0,1064.0,982.0,881.0,1486.0,949.0,575.0,414.0,1529.0
Hindu,9.0,48.0,7.0,9.0,11.0,34.0,47.0,1.0,54.0,37.0
Historically Black Prot,244.0,81.0,236.0,238.0,197.0,223.0,131.0,228.0,78.0,339.0
Jehovah's Witness,27.0,11.0,24.0,24.0,21.0,30.0,15.0,20.0,6.0,37.0
Jewish,19.0,87.0,25.0,25.0,30.0,95.0,69.0,19.0,151.0,162.0


### <span style='color:red'> Hands-on exercise </span>

For the `relinc.csv` dataset, plot the *frequency of income groups* for the religious affiliation `Agnostic` as *Barchart*. Execute the plot on the following data:

1. Plot based on the tidy format `df_melt_relinc`
2. Plot based on the original format `df_relinc`

Which data structure makes plotting easier for you?

### Multiple variables are stored in one column

After the melt process, we often receive variable names that represent a combination of several variables. This is demonstrated by the following data:

In [8]:
df_tb = pd.read_csv('./data/tb2.csv')

In [9]:
df_tb.sample(10)

,country,year,m014,m1524,m2534,m3544,m4554,m5564,m65,mu,f014,f1524,f2534,f3544,f4554,f5564,f65,fu
4252,PS,1980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5118,TM,1999,5.0,129.0,225.0,174.0,77.0,43.0,17.0,NaN,2.0,51.0,103.0,65.0,32.0,27.0,14.0,NaN
3629,MX,2005,100.0,1095.0,1376.0,1314.0,1238.0,1042.0,1288.0,NaN,125.0,771.0,733.0,710.0,784.0,637.0,784.0,NaN
5219,TT,1986,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2049,GU,2007,0.0,0.0,0.0,2.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,1.0,1.0,1.0,NaN
2342,IN,1986,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4434,RU,1991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4220,PL,2005,3.0,109.0,199.0,389.0,639.0,292.0,310.0,0.0,3.0,95.0,142.0,112.0,151.0,63.0,316.0,0.0
4273,PS,2005,NaN,1.0,NaN,NaN,1.0,3.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN
1575,EG,2006,54.0,542.0,728.0,563.0,587.0,340.0,136.0,NaN,64.0,470.0,367.0,338.0,279.0,155.0,87.0,NaN


This WHO data set on tuberculosis shows the number of tuberculosis patients per year for different countries in a demographic group. The demographic groups are made up of *gender* (`m`,`f`) and *age* (`0-14`,`15-25`, `25-34`, `35-44`, `45-54`, `55-64`, `unknown`). We now want to convert the data into a tidy format:

In [10]:
df_tb_melt = df_tb.melt(id_vars=['country', 'year'], var_name='column', value_name='cases')

In [11]:
df_tb_melt.head()

,country,year,column,cases
0,AD,1989,m014,NaN
1,AD,1990,m014,NaN
2,AD,1991,m014,NaN
3,AD,1992,m014,NaN
4,AD,1993,m014,NaN


Now we have to split the column with the combined variables:

In [12]:
df_tb_melt['gender'] = df_tb_melt['column'].str[0]

In [13]:
df_tb_melt['age'] = df_tb_melt['column'].str[1:].astype(str)

In [14]:
df_tb_melt.drop('column', axis='columns', inplace=True)

In [15]:
df_tb_melt.sample(10)

,country,year,cases,gender,age
30957,HK,1983,NaN,m,5564
15466,NR,2000,NaN,m,2534
76879,GH,1983,NaN,f,5564
28794,ZM,1985,NaN,m,4554
6566,BW,1993,NaN,m,1524
47313,CO,1985,NaN,f,014
8034,ID,1996,781.0,m,1524
39185,SD,1993,NaN,m,65
67846,QA,2006,7.0,f,3544
22023,SL,1994,NaN,m,3544


The data is now completely available as Tidy Data! We can just "tidy" it up a little more:

In [16]:
df_tb_melt['gender'] = df_tb_melt['gender'].map({'f': 'Female', 'm' : 'Male'})

In [17]:
df_tb_melt['age'] = df_tb_melt['age'].apply(lambda x : x[:2]+'-'+x[2:] if len(x) == 4 else x)

In [18]:
df_tb_melt.replace({'age' : {'u' : 'unknown', '014' : '0-14', '65' : '65+'}}, inplace=True)

In [19]:
df_tb_melt.sample(10)

,country,year,cases,gender,age
48594,IS,1999,0.0,Female,0-14
58615,CD,2005,8501.0,Female,25-34
25757,KI,2008,10.0,Male,45-54
56861,TC,1992,NaN,Female,15-24
53149,CU,1994,NaN,Female,15-24
40735,AZ,2000,NaN,Male,unknown
41429,CK,1984,NaN,Male,unknown
30389,EE,2004,14.0,Male,55-64
17968,BN,2000,15.0,Male,35-44
90686,PH,1993,NaN,Female,unknown


### Variables are stored in both rows and columns

It becomes complicated if the variables have been saved in both the rows and the columns:

In [20]:
df_wd = pd.read_csv('./data/weather-raw.csv')

In [21]:
df_wd

,id,year,month,element,d1,d2,d3,d4,d5,d6,d7,d8
0,MX17004,2010,1,tmax,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MX17004,2010,1,tmin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MX17004,2010,2,tmax,NaN,27.3,24.1,NaN,NaN,NaN,NaN,NaN
3,MX17004,2010,2,tmin,NaN,14.4,14.4,NaN,NaN,NaN,NaN,NaN
4,MX17004,2010,3,tmax,NaN,NaN,NaN,NaN,32.1,NaN,NaN,NaN
5,MX17004,2010,3,tmin,NaN,NaN,NaN,NaN,14.2,NaN,NaN,NaN
6,MX17004,2010,4,tmax,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,MX17004,2010,4,tmin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,MX17004,2010,5,tmax,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,MX17004,2010,5,tmin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


These are daily weather data from a weather station (MX17004) in Mexico over a period of 5 months. The days `d9` to `d31` have been removed for simplicity. For this data, variables exist in individual columns (`id`, `year` and `month`), across multiple columns (`day`, `d1-d8`) and in rows (`tmin`, `tmax`). The `element` column is not a variable, but contains the names of variables.

To bring this data into a tidy format, we "melt" the column variables `id`, `year` and `month`, as well as `element`.

In [22]:
df_wd_melt = df_wd.melt(id_vars=['id', 'year', 'month', 'element'], var_name='day', value_name='temp'); df_wd_melt

,id,year,month,element,day,temp
0,MX17004,2010,1,tmax,d1,NaN
1,MX17004,2010,1,tmin,d1,NaN
2,MX17004,2010,2,tmax,d1,NaN
3,MX17004,2010,2,tmin,d1,NaN
4,MX17004,2010,3,tmax,d1,NaN
...,...,...,...,...,...,...
75,MX17004,2010,3,tmin,d8,NaN
76,MX17004,2010,4,tmax,d8,NaN
77,MX17004,2010,4,tmin,d8,NaN
78,MX17004,2010,5,tmax,d8,NaN


The date information is now spread across 3 columns. We can display this more clearly in one column:

Extract of the day:

In [23]:
df_wd_melt['day'] = df_wd_melt['day'].str[1:]

Convert to a Date column:

In [24]:
df_wd_melt['date'] = pd.to_datetime(df_wd_melt.loc[:,['year', 'month', 'day']])

Drop the superfluous columns:

In [25]:
df_wd_melt.drop(['year', 'month', 'day'], axis='columns', inplace=True)

Reorder columns:

In [26]:
df_wd_melt = df_wd_melt[['id', 'date', 'element', 'temp']]

In [27]:
df_wd_melt.head()

,id,date,element,temp
0,MX17004,2010-01-01,tmax,NaN
1,MX17004,2010-01-01,tmin,NaN
2,MX17004,2010-02-01,tmax,NaN
3,MX17004,2010-02-01,tmin,NaN
4,MX17004,2010-03-01,tmax,NaN


This DataFrame is *almost* tidy. If you take `tmax` and `tmin` as individual variables (ergo: one day as observation per row, principle 2), then these must be displayed in separate columns again. In concrete terms, we must therefore pivot:

In [28]:
df_wd_melt.pivot_table(index=['id', 'date'],columns=['element'], values='temp', dropna=False).reset_index().head()

element,id,date,tmax,tmin
0,MX17004,2010-01-01,NaN,NaN
1,MX17004,2010-01-02,NaN,NaN
2,MX17004,2010-01-03,NaN,NaN
3,MX17004,2010-01-04,NaN,NaN
4,MX17004,2010-01-05,NaN,NaN


The DataFrame is now tidy and each day corresponds to an observation with the measured variables `tmin` and `tmax`.

### Multiple types of observational units are stored in the same table

In the following, we will look at a violation of the third principle:

> Each type of observational unit forms a table

Data sets often include values collected at multiple levels and contain different types of observations. Through tidying, each type of observation should be stored in its own table.  This is closely related to the idea of database normalization, where each fact is expressed in only one place.  If this is not done, inconsistencies can occur.

The following data represents the Billboard charts of the top 100 hits in 2000. The variables are `artist`, `track`, `date entered`, `date peaked`, `genre`, `time`, `rank` and `week`. The place in the charts (rank) of a song after it enters the Top 100 is recorded week by week in the columns `x1st.week` to `x76th.week`. If the song leaves the Top 100 in the meantime, the remaining values are filled with NaN.

In [29]:
df_bill = pd.read_csv('./data/billboard.csv',header=0,encoding = 'unicode_escape')

In [30]:
df_bill.head()

,year,artist.inverted,track,time,genre,date.entered,date.peaked,x1st.week,x2nd.week,x3rd.week,...,x67th.week,x68th.week,x69th.week,x70th.week,x71st.week,x72nd.week,x73rd.week,x74th.week,x75th.week,x76th.week
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23,2000-11-18,78,63.0,49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000,Santana,"Maria, Maria",4:18,Rock,2000-02-12,2000-04-08,15,8.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,1999-10-23,2000-01-29,71,48.0,43.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000,Madonna,Music,3:45,Rock,2000-08-12,2000-09-16,41,23.0,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,2000-08-05,2000-10-14,57,47.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Two types of observations can be identified in this dataset: the song itself and its rank per week. As a consequence, the Artist, Time and Genre are repeated for each week. This happens after we have extracted the values for the weeks from the columns:

In [31]:
df_bill_melt = df_bill.melt(id_vars=['year','artist.inverted','track','time','genre','date.entered','date.peaked'],
                            var_name='week', value_name='rank')
df_bill_melt.head()

,year,artist.inverted,track,time,genre,date.entered,date.peaked,week,rank
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23,2000-11-18,x1st.week,78.0
1,2000,Santana,"Maria, Maria",4:18,Rock,2000-02-12,2000-04-08,x1st.week,15.0
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,1999-10-23,2000-01-29,x1st.week,71.0
3,2000,Madonna,Music,3:45,Rock,2000-08-12,2000-09-16,x1st.week,41.0
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,2000-08-05,2000-10-14,x1st.week,57.0


This gives us a row in the DataFrame for each observation of the chart placement. We can now tidy up the date a bit more by calculating the date for each week from the week number and the `date.entered`. To do this, we must first extract the week number:

In [32]:
df_bill_melt['week'] = df_bill_melt['week'].str.findall(r'\d+').str[0]

We then convert the weeks into a `timedelta` object so that we can simply add up the times:

In [33]:
import datetime

In [34]:
df_bill_melt['date.entered'] = pd.to_datetime(df_bill_melt['date.entered'])

In [35]:
df_bill_melt['week_delta'] = df_bill_melt['week'].astype('int64').apply(lambda x : datetime.timedelta(weeks=(x-1)))

In [36]:
df_bill_melt['date'] = df_bill_melt['date.entered'] + df_bill_melt['week_delta']

We can now remove the redundant columns:

In [37]:
df_bill_melt.drop('date.entered', axis='columns', inplace=True)
df_bill_melt.drop('week', axis='columns', inplace=True)
df_bill_melt.drop('week_delta', axis='columns', inplace=True)
df_bill_melt.drop('date.peaked', axis='columns', inplace=True)

In [38]:
df_bill_melt.head()

,year,artist.inverted,track,time,genre,rank,date
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,78.0,2000-09-23
1,2000,Santana,"Maria, Maria",4:18,Rock,15.0,2000-02-12
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,71.0,1999-10-23
3,2000,Madonna,Music,3:45,Rock,41.0,2000-08-12
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,57.0,2000-08-05


In order to save the songs and their rank in two different DataFrames, we must assign a unique ID - otherwise no assignment can be made. We can achieve this with `pd.factorize`:

In [39]:
labels, unique = pd.factorize(df_bill_melt['track'])

In [40]:
df_bill_melt['id'] = labels+1

In [41]:
df_bill_melt.head()

,year,artist.inverted,track,time,genre,rank,date,id
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,78.0,2000-09-23,1
1,2000,Santana,"Maria, Maria",4:18,Rock,15.0,2000-02-12,2
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,71.0,1999-10-23,3
3,2000,Madonna,Music,3:45,Rock,41.0,2000-08-12,4
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,57.0,2000-08-05,5


Now we can split our DataFrame and separate the observations *Song* and *Ranking*:

In [42]:
df_rank = df_bill_melt[['id', 'date', 'rank']].copy()

In [43]:
df_rank.sort_values(by=['id', 'date']).reset_index(drop=True).head()

,id,date,rank
0,1,2000-09-23,78.0
1,1,2000-09-30,63.0
2,1,2000-10-07,49.0
3,1,2000-10-14,33.0
4,1,2000-10-21,23.0


In [44]:
df_song = df_bill_melt[['id', 'artist.inverted', 'track', 'time']].copy()
df_song.drop_duplicates('id', inplace=True)

In [45]:
df_song.head()

,id,artist.inverted,track,time
0,1,Destiny's Child,Independent Women Part I,3:38
1,2,Santana,"Maria, Maria",4:18
2,3,Savage Garden,I Knew I Loved You,4:07
3,4,Madonna,Music,3:45
4,5,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38


### A single observational unit is stored in multiple tables

We have already learned enough about how to tackle this problem in the learning unit on `pd.concat` and `pd.merge`. Therefore, here is the concatenation of two DataFrames with identical variables:

In [46]:
df_baby14 = pd.read_csv("./data/2014-baby-names-illinois.csv")
df_baby15 = pd.read_csv("./data/2015-baby-names-illinois.csv")

In [47]:
df_baby14.head()

,rank,name,frequency,sex
0,1,Noah,837,Male
1,2,Alexander,747,Male
2,3,William,687,Male
3,4,Michael,680,Male
4,5,Liam,670,Male


In [48]:
df_baby14.shape

(101, 4)

In [49]:
df_baby15.head()

,rank,name,frequency,sex
0,1,Noah,863,Male
1,2,Liam,709,Male
2,3,Alexander,703,Male
3,4,Jacob,650,Male
4,5,William,618,Male


In [50]:
df_baby15.shape

(100, 4)

In [51]:
df_baby = pd.concat([df_baby14, df_baby15]).sort_values(by=['rank'])

In [52]:
df_baby.shape

(201, 4)

## Extract observations from lists with `explode()`
---

Data often contains different characteristic values as list elements in one column. If you want to analyze such a variable, the list must be exploded. We can achieve this with [explode()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.explode.html), here using the IMDB data:

In [53]:
df_imdb = pd.read_csv('./data/imdb_ratings.csv')

In [54]:
df_imdb.head()

,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76.0
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,126.46,65.0
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,138.12,62.0
3,4,Sing,"Animation,Comedy,Family","In a city of humanoid animals, a hustling thea...",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",2016,108,7.2,60545,270.32,59.0
4,5,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,123,6.2,393727,325.02,40.0


For example, if we want to analyze the *average sales per genre*, then we need to declare the genres as *observations*. According to Tidy Data principles, each observation should be in one line. To do this, we must first convert the string into a list:

In [55]:
df_imdb['genre_list'] = df_imdb['Genre'].str.split(',')

In [56]:
df_imdb['genre_list'][0]

['Action', 'Adventure', 'Sci-Fi']

With `explode()` we can now explode this list into a `long format`. This means that we receive a new line with the corresponding index element for each list entry and therefore considerably enlarge our DataFrame. This would correspond to an $m \times n$ table:

In [57]:
df_imdb_expl = df_imdb.explode('genre_list')

In [58]:
df_imdb_expl.head()

,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore,genre_list
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76.0,Action
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76.0,Adventure
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76.0,Sci-Fi
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,126.46,65.0,Adventure
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,126.46,65.0,Mystery


The naming of the columns is no longer appropriate and can be changed:

In [59]:
df_imdb_expl.rename(columns={'Genre' : 'genre_list', 'genre_list' : 'genre'}, inplace=True)

Finally, we can group according to the individual genres:

In [60]:
df_imdb_expl[['genre', 'Revenue (Millions)']].groupby('genre').agg('mean')

,Revenue (Millions)
genre,
Action,124.494476
Adventure,154.177024
Animation,191.223404
Biography,55.801600
Comedy,75.750784
Crime,61.804769
Drama,49.844205
Family,126.175714
Fantasy,131.850108


## Learning outcomes
---

If you have carefully studied the materials provided in this document, you should have a reasonable understanding of how use tidy data principles to prepare and reshape datasets for convenient analysis.

This understanding includes:

* the principles of tidy data as well as the terminology,
* implementing tidy data principles in Pandas, especially for the following violations:
    * Column names are values and not variables,
    * Several variables are stored in one column,
    * Variables are saved in rows and columns,
    * Several types of observations are saved in the same table,
    * A single type of observation is stored in several tables,
* and how to extract observations from lists.